# Training Pipeline
[run_training_dpo_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [1]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

fatal: destination path 'MedicalGPT' already exists and is not an empty directory.
/content/MedicalGPT
build_domain_tokenizer.py   README.md
chatpdf.py                  requirements.txt
CITATION.cff                reward_modeling.py
_config.yml                 role_play_data/
CONTRIBUTING.md             run_dpo.sh
convert_dataset.py          run_eval_quantize.sh
data/                       run_full_sft.sh
DISCLAIMER                  run_grpo.sh
docs/                       run_orpo.sh
dpo_training.py             run_ppo.sh
eval_quantize.py            run_pt.sh
fastapi_server_demo.py      run_quant.sh
gradio_demo.py              run_rm.sh
grpo_training.py            run_sft_accelerate.sh
inference_multigpu_demo.py  run_sft.sh
inference.py                run_training_dpo_pipeline.ipynb
LICENSE                     run_training_ppo_pipeline.ipynb
MedicalGPT/                 supervised_finetuning_accelerate.py
merge_peft_adapter.py       supervised_finetuning.py
merge_tokenizers.py         t

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [2]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [3]:
!python pretraining.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --bf16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 50 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2025-12-21 05:38:34.726302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766295514.746307    6483 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766295514.752194    6483 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766295514.767272    6483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766295514.767297    6483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766295514.767301    6483 computation_placer.cc:177] computation placer alr

In [4]:
%ls -lh outputs-pt-v1

total 22M
-rw-r--r-- 1 root root 1.1K Dec 21 05:48 adapter_config.json
-rw-r--r-- 1 root root  17M Dec 21 05:48 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Dec 21 05:48 added_tokens.json
-rw-r--r-- 1 root root  472 Dec 21 05:48 all_results.json
-rw-r--r-- 1 root root 2.4K Dec 21 05:48 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Dec 21 05:47 checkpoint-750/
drwxr-xr-x 2 root root 4.0K Dec 21 05:47 checkpoint-800/
drwxr-xr-x 2 root root 4.0K Dec 21 05:48 checkpoint-834/
-rw-r--r-- 1 root root  263 Dec 21 05:48 eval_results.json
-rw-r--r-- 1 root root 1.6M Dec 21 05:48 merges.txt
-rw-r--r-- 1 root root 5.1K Dec 21 05:48 README.md
drwxr-xr-x 4 root root 4.0K Dec 21 05:38 runs/
-rw-r--r-- 1 root root  616 Dec 21 05:48 special_tokens_map.json
-rw-r--r-- 1 root root 4.7K Dec 21 05:48 tokenizer_config.json
-rw-r--r-- 1 root root  20K Dec 21 05:48 trainer_state.json
-rw-r--r-- 1 root root  229 Dec 21 05:48 train_results.json
-rw-r--r-- 1 root root 3.3M Dec 21 05:48 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [5]:
!python merge_peft_adapter.py \
    --base_model Qwen/Qwen2.5-0.5B --lora_model outputs-pt-v1 --output_dir merged-pt/

2025-12-21 05:49:37.848537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766296177.868097    9278 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766296177.874359    9278 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766296177.890336    9278 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766296177.890359    9278 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766296177.890363    9278 computation_placer.cc:177] computation placer alr

In [6]:
%ls -lh merged-pt/

total 958M
-rw-r--r-- 1 root root  605 Dec 21 05:49 added_tokens.json
-rw-r--r-- 1 root root 2.4K Dec 21 05:49 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Dec 21 05:49 config.json
-rw-r--r-- 1 root root  117 Dec 21 05:49 generation_config.json
-rw-r--r-- 1 root root 1.6M Dec 21 05:49 merges.txt
-rw-r--r-- 1 root root 943M Dec 21 05:49 model.safetensors
-rw-r--r-- 1 root root  616 Dec 21 05:49 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 21 05:49 tokenizer_config.json
-rw-r--r-- 1 root root  11M Dec 21 05:49 tokenizer.json
-rw-r--r-- 1 root root 2.7M Dec 21 05:49 vocab.json


In [7]:
%cat merged-pt/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [8]:
%ls ./data/finetune

medical_sft_1K_format.jsonl        sharegpt_zh_1K_format.jsonl
numina_cot_sharegpt_data_1k.jsonl


In [10]:
!mv ./data/finetune/numina_cot_sharegpt_data_1k.jsonl ./data/finetune/numina_cot_sharegpt_data_1k.jsonl.bak
!cp ./data/finetune/numina_cot_sharegpt_data_1k.strict.jsonl ./data/finetune/numina_cot_sharegpt_data_1k.jsonl
!ls -lh ./data/finetune/numina_cot_sharegpt_data_1k.jsonl*

cp: cannot stat './data/finetune/numina_cot_sharegpt_data_1k.strict.jsonl': No such file or directory
-rw-r--r-- 1 root root 1.6M Dec 21 05:20 ./data/finetune/numina_cot_sharegpt_data_1k.jsonl.bak


In [11]:
!python supervised_finetuning.py \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --bf16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2025-12-21 05:52:40.896955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766296360.939210   10094 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766296360.949872   10094 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766296360.972786   10094 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766296360.972816   10094 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766296360.972821   10094 computation_placer.cc:177] computation placer alr

In [12]:
%ls -lh outputs-sft-v1

total 22M
-rw-r--r-- 1 root root 1.1K Dec 21 06:04 adapter_config.json
-rw-r--r-- 1 root root  17M Dec 21 06:04 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Dec 21 06:04 added_tokens.json
-rw-r--r-- 1 root root  429 Dec 21 06:04 all_results.json
-rw-r--r-- 1 root root 2.4K Dec 21 06:04 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Dec 21 06:04 checkpoint-250/
-rw-r--r-- 1 root root  220 Dec 21 06:04 eval_results.json
-rw-r--r-- 1 root root 1.6M Dec 21 06:04 merges.txt
-rw-r--r-- 1 root root 5.1K Dec 21 06:04 README.md
drwxr-xr-x 3 root root 4.0K Dec 21 05:53 runs/
-rw-r--r-- 1 root root  648 Dec 21 06:04 special_tokens_map.json
-rw-r--r-- 1 root root 4.7K Dec 21 06:04 tokenizer_config.json
-rw-r--r-- 1 root root 6.0K Dec 21 06:04 trainer_state.json
-rw-r--r-- 1 root root  229 Dec 21 06:04 train_results.json
-rw-r--r-- 1 root root 3.3M Dec 21 06:04 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [13]:
!python merge_peft_adapter.py \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir ./merged-sft

2025-12-21 06:05:54.820829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766297154.840181   13494 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766297154.846079   13494 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766297154.861332   13494 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766297154.861356   13494 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766297154.861360   13494 computation_placer.cc:177] computation placer alr

In [14]:
%ls -lh merged-sft/

total 958M
-rw-r--r-- 1 root root  605 Dec 21 06:06 added_tokens.json
-rw-r--r-- 1 root root 2.4K Dec 21 06:06 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Dec 21 06:06 config.json
-rw-r--r-- 1 root root  117 Dec 21 06:06 generation_config.json
-rw-r--r-- 1 root root 1.6M Dec 21 06:06 merges.txt
-rw-r--r-- 1 root root 943M Dec 21 06:06 model.safetensors
-rw-r--r-- 1 root root  616 Dec 21 06:06 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 21 06:06 tokenizer_config.json
-rw-r--r-- 1 root root  11M Dec 21 06:06 tokenizer.json
-rw-r--r-- 1 root root 2.7M Dec 21 06:06 vocab.json


In [15]:
%cat merged-sft/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage2 SFT训练完成。

# Stage 3: DPO(Direct Preference Optimization)

第三阶段：DPO(Direct Preference Optimization)直接偏好优化，DPO通过直接优化语言模型来实现对其行为的精确控制，而无需使用复杂的强化学习，也可以有效学习到人类偏好，DPO相较于RLHF更容易实现且易于训练，效果更好

| Stage 3: Direct Preference Optimization        |  [dpo_training.py](https://github.com/shibing624/MedicalGPT/blob/main/dpo_training.py) | [run_dpo.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_dpo.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是`Qwen/Qwen2.5-0.5B` 或者 Stage2得到的SFT模型
2. 数据集：DPO阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [16]:
%ls ./data/reward/

dpo_zh_500.jsonl


In [18]:
!python dpo_training.py \
    --model_name_or_path ./merged-sft \
    --template_name qwen \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 500 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 50 \
    --max_source_length 256 \
    --max_target_length 256 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --bf16 True \
    --fp16 False \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache \
    --optim adamw_torch

2025-12-21 06:08:08.110886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766297288.130376   14141 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766297288.136314   14141 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766297288.152548   14141 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766297288.152573   14141 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766297288.152579   14141 computation_placer.cc:177] computation placer alr

In [19]:
%ls -lh outputs-dpo-v1

total 22M
-rw-r--r-- 1 root root 1.1K Dec 21 06:36 adapter_config.json
-rw-r--r-- 1 root root  17M Dec 21 06:36 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Dec 21 06:36 added_tokens.json
-rw-r--r-- 1 root root  767 Dec 21 06:37 all_results.json
-rw-r--r-- 1 root root 2.4K Dec 21 06:36 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Dec 21 06:36 checkpoint-100/
drwxr-xr-x 2 root root 4.0K Dec 21 06:22 checkpoint-50/
-rw-r--r-- 1 root root  572 Dec 21 06:37 eval_results.json
-rw-r--r-- 1 root root 1.6M Dec 21 06:36 merges.txt
-rw-r--r-- 1 root root 2.4K Dec 21 06:36 README.md
drwxr-xr-x 3 root root 4.0K Dec 21 06:08 runs/
-rw-r--r-- 1 root root  648 Dec 21 06:36 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 21 06:36 tokenizer_config.json
-rw-r--r-- 1 root root  57K Dec 21 06:36 trainer_state.json
-rw-r--r-- 1 root root 6.7K Dec 21 06:36 training_args.bin
-rw-r--r-- 1 root root  229 Dec 21 06:36 train_results.json
-rw-r--r-- 1 root root 3.3M Dec 21 06:36 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [20]:
!python merge_peft_adapter.py \
    --base_model merged-sft --lora_model outputs-dpo-v1 --output_dir merged-dpo/

2025-12-21 06:40:45.494878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766299245.521486   22250 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766299245.527558   22250 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766299245.542954   22250 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766299245.542981   22250 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766299245.542996   22250 computation_placer.cc:177] computation placer alr

In [21]:
%ls -lh merged-dpo/

total 958M
-rw-r--r-- 1 root root  605 Dec 21 06:40 added_tokens.json
-rw-r--r-- 1 root root 2.4K Dec 21 06:40 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Dec 21 06:40 config.json
-rw-r--r-- 1 root root  117 Dec 21 06:40 generation_config.json
-rw-r--r-- 1 root root 1.6M Dec 21 06:40 merges.txt
-rw-r--r-- 1 root root 943M Dec 21 06:41 model.safetensors
-rw-r--r-- 1 root root  616 Dec 21 06:40 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 21 06:40 tokenizer_config.json
-rw-r--r-- 1 root root  11M Dec 21 06:40 tokenizer.json
-rw-r--r-- 1 root root 2.7M Dec 21 06:40 vocab.json


In [22]:
%cat merged-dpo/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage3 偏好建模第一次训练完成。

**至此一个完整的训练流程演示完成。**

# Test

In [23]:
!python inference.py --base_model merged-dpo
# 或在shell中运行
# python inference.py --base_model merged-dpo --interactive

2025-12-21 06:43:01.595726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766299381.615785   22842 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766299381.621872   22842 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766299381.637029   22842 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766299381.637055   22842 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766299381.637059   22842 computation_placer.cc:177] computation placer alr

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
